# Программы с фильтром по названию и времени выхода
Пример выгрузки всех выходов программы с условием отбора по ее названию и времени выхода

## Описание задачи и условий расчета
- Период: 01.09.2022-30.09.2022, рабочие дни
- ЦА: Все 4+ в разбивке на Федеральный округ
- Место просмотра: Все места (Дом+Дача)
- Каналы: ТНТ
- Программы: СашаТаня, Время начала – 17:50-18:10
- Статистики: Rtg%, Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники
Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# Получим id телекомпании
cats.get_tv_company(name=["ТНТ (СЕТЕВОЕ ВЕЩАНИЕ)"])

#Для ТНТ это 1870

In [ ]:
# По названию программы найдем ее id
cats.get_tv_program(name=["СашаТаня"])

# В расчете будем использовать id 224106

In [ ]:
# в справочинике типов дней найдем идентификатор рабочих дней
cats.get_tv_research_day_type()

# В расчете будем использовать id W

In [ ]:
# В словаре демографических атрибутов найдем переменную Федеральные округа
cats.find_tv_property('Федеральные округа')

# В условиях расчета будем использовать entityName переменной: federalOkrug

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2022-09-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня - рабочие
daytype_filter = 'researchDayType = W'

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы - ТНТ
company_filter = 'tvCompanyId = 1870'

# Указываем фильтр программ: программа СашаТаня, время начала от 17:50 до 18:10  
program_filter = 'programId = 224106 and programStartTime >= 175000 and programStartTime <= 181000'

# Фильтр блоков
break_filter = None

# Фильтр роликов
ad_filter = None

# Указываем список срезов
slices = ['programSpotId', #Программа ID выхода, обязательный атрибут! 
          'researchDate', #Дата, обязательный атрибут! 
          'programName', #Программа
          'tvCompanyName', #Телекомпания
          'researchWeekDay', #День недели
          'programStartTime', #Программа время начала
          'programFinishTime', #Программа время окончания
          'programCategoryName', #Программа категория
          'programIssueDescriptionName', #Программа описание выпуска
          'programProducerYear', #Программа дата создания
          'federalOkrug' #Разбивка на Федеральные округа
         ]
# Указываем список статистик для расчета
statistics = ['RtgPer', 'Share']

# Задаем условия сортировки: телекомпания (от а до я), дата (от старых к новым), время начала (по возр.)
sortings = {'tvCompanyName':'ASC', 'researchDate':'ASC', 'programStartTime':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1 #TV Index Russia all
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_simple_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]
df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('02_name_filter')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)